# Lab 9: Spatial-Temporal patterns of Natural Disturbance
Building on the ForestFire model from previous lab, compute the fire iternval distribution (times between fire intervals) and the patch size distribution (size of forested patches in steady-state landscape pattern).

Plot these data on a log-log using the techniques from Ch. 10 to see if the distribution appears to be "heavy tailed", and thus if this system exhibits properties of self-organizing criticality.


In [ ]:
!pip install empiricaldist

import numpy as np
from matplotlib import pyplot as plt
from empiricaldist import Pmf

### Create Continuous Patches
This is a surprisingly challenging problem to solve in the general case given how good our visual system is at identifying them!
The idea I had here was to start by giving each occupied cell a unique value, then "grow" patches from occupied cells by allowing the smallest of these unique values to propogate to neighbouring cells.  Repeat until the propogation is finished.


In [ ]:
neighbourhood = np.array([
    [0, 1, 0],
    [1, 1, 1],
    [0, 1, 0],
])

def min_neighbour(a):
    """ Return the smallest non-zero neighbourhood value or 0 if centre cell is a zero """
    p = a*neighbourhood
    centre = tuple(d//2 for d in a.shape)
    return np.min(p[p>0]) if a[centre] else 0

def consolidate(array):
    """ return copy of array with adjacent cells consolidated into a patch with the lowest value among occupied neighbours """
    rows, cols = array.shape
    k = neighbourhood.shape[0]
    array = np.pad(array, 1, 'constant')
    return np.array([
        [min_neighbour(array[row:row+k, col:col+k]) for col in range(cols) ]
            for row in range(rows)
    ])

def patchify(array, category):
    """ Return an array with each contiguous patch identified by a unique integer
    array:  array of int categorical values
    category: the int category value to identify patches

    return: array of same shape with a unique value identifying cells in each patch and zeros elsewhere
    """
    patches = np.zeros(array.shape, dtype=np.uint)
    patches[array==category] = range(1, len(array[array==category])+1)

    patches_growing = np.array([True,])
    while np.any(patches_growing):
        prev_patches = patches
        patches = consolidate(prev_patches)
        patches_growing = patches != prev_patches  # patches are growning until consolidate algorithm stablaizes.

    return patches

## Code fragments...

In [ ]:
# Create an array of patches from occupied cells of a forest array
patches = patchify(forest.array, OCCUPIED)
draw_array(patches, cmap='Greens', vmin=0, vmax=np.max(patches))

In [ ]:
def plot_patch_sizes(patch_sizes, min_size=1, scale='linear', plot_type='bar'):
    """ plot the distribution of patch sizes for the array of patch sizes """
    plot_options = dict(xlabel='patch size', ylabel='N patches', xscale=scale, yscale=scale)
    fig, ax = plt.subplots(figsize=(6, 6), subplot_kw=plot_options)
    ax.set_title("Patch Size Distribution")

    # get unique patch size classes with count of patches in each size class
    size_classes, counts = np.unique(patch_sizes[patch_sizes>=min_size], return_counts=True)
    if plot_type == 'bar' and scale == 'linear':
        ax.bar(size_classes, counts)
    else:
        ax.plot(size_classes, counts)

    n_patches = len(patch_sizes)
    print('Number of patches:', n_patches, 'Unique patch size classes:', len(size_classes))
    single_cell_patches = np.sum(patch_sizes[patch_sizes==1])
    print('Number of single cell patches:', single_cell_patches, '({pct}%)'.format(pct=round(100*single_cell_patches/n_patches)))
    print('Largest patch size:', np.max(patch_sizes))

In [ ]:
# get list of unique patches with count of cells in each patch
patch_ids, patch_sizes = np.unique(patches[patches>0], return_counts=True)
# lot the patch size distribution as a bar chart
plot_patch_sizes(patch_sizes, )